In [0]:
import tensorflow as tf
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from random import randint
import random
from sklearn import preprocessing

iris = datasets.load_iris()
X = iris.data
y = iris.target


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [0]:
#Neural network module
from keras.models import Sequential 
from keras.layers import Dense,Activation,Dropout 
from keras.layers.normalization import BatchNormalization 
from keras.utils import to_categorical

In [0]:
def initialize_population(population_size):
    activation = ['tanh', 'sigmoid', 'relu']
    optimizer = ['sgd', 'rmsprop', 'adam']
    pop = np.array([[random.choice(activation), random.choice(optimizer), randint(2,100),randint(2,50)]])
    for i in range(0, population_size-1):
        pop = np.append(pop, [[random.choice(activation), random.choice(optimizer), randint(2,100),randint(2,50)]], axis = 0)
    return pop

In [0]:
def crossover(parent_1, parent_2):
    child = [parent_1[0], parent_2[1], parent_1[2], parent_2[3]]    
    return child

In [0]:
def mutate(child, prob_mut):
    child_ = np.copy(child)
    for c in range(0, len(child_)):
        if np.random.rand() >= prob_mut:
            k = randint(2,3)
            child_[c,k] = int(child_[c,k]) + randint(1, 4)
    return child_

In [0]:
def function_fitness_keras(pop, X_train, y_train, X_test, y_test):
    fitness = []
    j = 0
    for w in pop:
        
        model = Sequential()
        model.add(Dense(int(w[2]), input_dim = 4, activation = w[0]))
        model.add(Dense(int(w[3]), activation = w[0]))
        model.add(Dropout(0.2))
        model.add(Dense(3, activation = 'softmax'))
        model.compile(loss = 'categorical_crossentropy', optimizer = w[1], metrics = ['accuracy'])
        
        try:
            print('Model ', j, ':')
            j = j + 1
        
            model.fit(X_train, y_train, batch_size = 20, epochs = 10, verbose = 0)
            y_label = np.argmax(y_test, axis = 1)
            predict_label = model.predict_classes(X_test)
            f = np.sum(predict_label == y_label) / len(y_test)
        
            print(f)
            fitness.append([f, model, w])
        except:
            pass
    return fitness

In [0]:
def ga_keras(X_train, y_train, X_test, y_test, num_epochs = 10, population_size = 10, prob_mut = 0.8):
    pop = initialize_population(population_size)
    fitness = function_fitness_keras(pop,  X_train, y_train, X_test, y_test)
    print('shape of fitness = ', np.shape(fitness))
    
    pop_fitness_sort = np.array(list(reversed(sorted(fitness,key=lambda x: x[0]))))

    print('shape of pop_fitness_sort = ', pop_fitness_sort.shape)
    
    for j in range(0, num_epochs):
        print('--------- Iteration ', j, ' ----------')
        
        length = len(pop_fitness_sort)
        # Selection of Parents
        parent_1 = pop_fitness_sort[:,2][:length//2]
        parent_2 = pop_fitness_sort[:,2][length//2:]

        print('Parent selection done')
        
        # Crossover and Mutation
        child_1 = [crossover(parent_1[i], parent_2[i]) for i in range(0, np.min([len(parent_2), len(parent_1)]))]
        child_2 = [crossover(parent_2[i], parent_1[i]) for i in range(0, np.min([len(parent_2), len(parent_1)]))]
        child_2 = mutate(child_2, prob_mut)
        
        print('Crossover and mutation done')
        
        # Calculation of fitness and sorting by best fitness to worst
        fitness_child_1 = function_fitness_keras(child_1,X_train, y_train, X_test, y_test)
        fitness_child_2 = function_fitness_keras(child_2, X_train, y_train, X_test, y_test)
        pop_fitness_sort = np.concatenate((pop_fitness_sort, fitness_child_1, fitness_child_2))
        sort = np.array(list(reversed(sorted(pop_fitness_sort,key=lambda x: x[0]))))
        
        print('Fitness calculated')
        
        # Selection of Best Individual, i.e., the fittest chromosome 
        pop_fitness_sort = sort[0:population_size, :]
        print('Best fitness = ', sort[0][0])
        best_individual = sort[0][1]
        
    return best_individual

In [0]:
# labels_to_categorical
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [0]:
# StandardScaler
scaler = preprocessing.StandardScaler()

In [0]:
# scale_features
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [33]:
best_model = ga_keras(X_train_normalized, y_train_cat, X_test_normalized, y_test_cat, num_epochs = 5, population_size = 10, prob_mut = 0.8)
# print (np.sum(best_model.predict(X_test_normalized) == y_test_cat) / len(y_test_cat)) # print accuracy of best_individual

Model  0 :
0.8666666666666667
Model  1 :
0.9
Model  2 :
0.8666666666666667
Model  3 :
0.6333333333333333
Model  4 :
0.9
Model  5 :
0.8
Model  6 :
0.3333333333333333
Model  7 :
0.7666666666666667
Model  8 :
0.3
Model  9 :
0.9
shape of fitness =  (10, 3)
shape of pop_fitness_sort =  (10, 3)
--------- Iteration  0  ----------
Parent selection done
Crossover and mutation done
Model  0 :
0.8666666666666667
Model  1 :
0.9
Model  2 :
0.9333333333333333
Model  3 :
0.7333333333333333
Model  4 :
0.5666666666666667
Model  0 :
0.26666666666666666
Model  1 :
0.7666666666666667
Model  2 :
0.6666666666666666
Model  3 :
0.7
Model  4 :
0.3333333333333333
Fitness calculated
Best fitness =  0.9333333333333333
--------- Iteration  1  ----------
Parent selection done
Crossover and mutation done
Model  0 :
0.9333333333333333
Model  1 :
0.9
Model  2 :
0.9
Model  3 :
0.9333333333333333
Model  4 :
0.4666666666666667
Model  0 :
0.9
Model  1 :
0.6
Model  2 :
0.8666666666666667
Model  3 :
0.3333333333333333
Model

In [34]:
y_label = np.argmax(y_test_cat, axis = 1)
predict_label = best_model.predict_classes(X_test_normalized)
f = np.sum(predict_label == y_label) / len(y_test_cat)
print(f)

0.9666666666666667


In [35]:
best_model.summary()

Model: "sequential_338"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1010 (Dense)           (None, 100)               500       
_________________________________________________________________
dense_1011 (Dense)           (None, 25)                2525      
_________________________________________________________________
dropout_337 (Dropout)        (None, 25)                0         
_________________________________________________________________
dense_1012 (Dense)           (None, 3)                 78        
Total params: 3,103
Trainable params: 3,103
Non-trainable params: 0
_________________________________________________________________
